# 🎓 AI Lab Final Exam: Complete Master Helper
**Instructions for the Exam:**
1. **Imports First:** Run the first code cell immediately.
2. **Identify Problem:** Is it search? logic? clustering? neural?
3. **Define Your Logic:** Find the specific algorithm cell and modify the 'Scenario Example' section to match the exam question.
4. **Run and Copy:** Get the path/solution/weights and move to the next question.

In [ ]:
# --- MANDATORY IMPORTS ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import heapq
import random
from collections import deque
from itertools import product
print('✅ Environment Ready.')

## SECTION 1: SEARCH ALGORITHMS (Labs 1-8)
### 1. BFS / DFS / UCS / A* (Generic Graph/Grid Solver)

In [ ]:
import heapq
from collections import deque

# =========================================================
# 1. THE ALGORITHMS (Copy-pasted from your provided code)
# =========================================================

def breadth_first_search(initial_state, is_goal, get_successors):
    frontier = deque([(initial_state, [initial_state])])
    explored = {initial_state}
    while frontier:
        current, path = frontier.popleft()
        if is_goal(current): return path
        for neighbor in get_successors(current):
            if neighbor not in explored:
                explored.add(neighbor)
                frontier.append((neighbor, path + [neighbor]))
    return None

def depth_first_search(initial_state, is_goal, get_successors):
    frontier = [(initial_state, [initial_state])]
    explored = set()
    while frontier:
        current, path = frontier.pop()
        if is_goal(current): return path
        if current not in explored:
            explored.add(current)
            for neighbor in get_successors(current):
                frontier.append((neighbor, path + [neighbor]))
    return None

def uniform_cost_search(initial_state, is_goal, get_successors, get_cost):
    frontier = [(0, initial_state, [initial_state])]
    explored = {} 
    while frontier:
        cost, current, path = heapq.heappop(frontier)
        if is_goal(current): return path, cost
        if current not in explored or cost < explored[current]:
            explored[current] = cost
            for neighbor in get_successors(current):
                new_cost = cost + get_cost(current, neighbor)
                heapq.heappush(frontier, (new_cost, neighbor, path + [neighbor]))
    return None
 
def a_star_search(initial_state, is_goal, get_successors, get_cost, heuristic):
    frontier = [(heuristic(initial_state), 0, initial_state, [initial_state])]
    explored = {}
    while frontier:
        f, g, current, path = heapq.heappop(frontier)
        if is_goal(current): return path, g
        if current not in explored or g < explored[current]:
            explored[current] = g
            for neighbor in get_successors(current):
                new_g = g + get_cost(current, neighbor)
                new_f = new_g + heuristic(neighbor)
                heapq.heappush(frontier, (new_f, new_g, neighbor, path + [neighbor]))
    return None

# =========================================================
# 2. THE SCENARIO (How to call them in an exam)
# =========================================================

# Imagine a 3x3 Grid. Each number is the 'elevation' (cost to enter that cell).
grid = [
    [1, 5, 1],
    [1, 9, 1],
    [1, 1, 1]
]

# A. SUCCESSOR FUNCTION: Where can we move from current state (r, c)?
def get_neighbors(state):
    r, c = state
    results = []
    for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]: # Right, Left, Down, Up
        nr, nc = r + dr, c + dc
        if 0 <= nr < 3 and 0 <= nc < 3: # Stay inside the 3x3 grid
            results.append((nr, nc))
    return results

# B. COST FUNCTION: What is the cost to move from s1 to s2?
def cost_fn(s1, s2):
    return grid[s2[0]][s2[1]] # Cost is the elevation value of the destination

# C. HEURISTIC: Estimated distance to goal (Manhattan Distance)
def manhattan_h(state):
    goal = (2, 2)
    return abs(state[0] - goal[0]) + abs(state[1] - goal[1])

# D. GOAL TEST: Are we at (2, 2)? 
is_at_goal = lambda s: s == (2, 2)
 
# =========================================================
# 3. EXECUTING AND DISPLAYING OUTPUT
# =========================================================

start_node = (0, 0)

print("--- BFS RESULT (Shortest path by number of steps) ---")
bfs_path = breadth_first_search(start_node, is_at_goal, get_neighbors)
print(f"Path: {bfs_path}\n")

print("--- DFS RESULT (Deepest path, often not optimal) ---")
dfs_path = depth_first_search(start_node, is_at_goal, get_neighbors)
print(f"Path: {dfs_path}\n")

print("--- UCS RESULT (Cheapest path based on grid costs) ---")
ucs_res = uniform_cost_search(start_node, is_at_goal, get_neighbors, cost_fn)
print(f"Path: {ucs_res[0]} | Total Cost: {ucs_res[1]}\n")

print("--- A* RESULT (Cheapest path guided by heuristic) ---")
astar_res = a_star_search(start_node, is_at_goal, get_neighbors, cost_fn, manhattan_h)
print(f"Path: {astar_res[0]} | Total Cost: {astar_res[1]}")

### 2. Hill Climbing (Local Search)
Used for Lab 8 Path Optimization.

In [ ]:
# YOUR EXACT FUNCTION
def hill_climbing(initial_state, get_neighbors, evaluate, restarts=0):
    best_state = initial_state
    for _ in range(restarts + 1):
        # If restart > 0, we could pick a random starting point (handled in the exam logic below)
        current = initial_state if _ == 0 else random_state_generator()
        while True:
            neighbors = get_neighbors(current)
            if not neighbors: break
            next_state = min(neighbors, key=evaluate) # Min for elevation cost
            if evaluate(next_state) >= evaluate(current): break
            current = next_state
        if evaluate(current) < evaluate(best_state): best_state = current
    return best_state, evaluate(best_state)

# --- REAL LAB 8 IMPLEMENTATION ---
terrain = [
    [9, 8, 7, 6],
    [6, 5, 4, 7],   
    [5, 3, 2, 5],
    [8, 7, 3, 1]
]

def drone_get_neighbors(state):
    r, c = state
    res = []
    for dr, dc in [(0,1), (0,-1), (1,0), (-1,0)]:
        if 0 <= r+dr < 4 and 0 <= c+dc < 4:
            res.append((r+dr, c+dc))
    return res

def drone_evaluate(state):
    return terrain[state[0]][state[1]]

def random_state_generator():
    return (random.randint(0, 3), random.randint(0, 3))

# Starting at top-left (0,0)
best_pos, min_elevation = hill_climbing((0,0), drone_get_neighbors, drone_evaluate, restarts=3)

print(f"Drone found local minimum at coordinates: {best_pos}")
print(f"Elevation cost at this point: {min_elevation}")

## SECTION 2: ADVERSARIAL SEARCH & CSP (Labs 9-10)
### 3. Alpha-Beta Pruning (Connect-4 / Tic-Tac-Toe / Chess)

In [ ]:
# YOUR EXACT FUNCTION
def alpha_beta(state, depth, alpha, beta, is_max, get_moves, evaluate, is_terminal):
    if depth == 0 or is_terminal(state): return evaluate(state), None
    best_move = None
    if is_max:
        v = float('-inf')
        for move, child in get_moves(state):
            score, _ = alpha_beta(child, depth-1, alpha, beta, False, get_moves, evaluate, is_terminal)
            if score > v: v, best_move = score, move
            alpha = max(alpha, v)
            if alpha >= beta: break
        return v, best_move
    else:
        v = float('inf')
        for move, child in get_moves(state):
            score, _ = alpha_beta(child, depth-1, alpha, beta, True, get_moves, evaluate, is_terminal)
            if score < v: v, best_move = score, move
            beta = min(beta, v)
            if alpha >= beta: break
        return v, best_move

# --- REAL LAB 8 IMPLEMENTATION: HARVEST DUEL ---
# Rule: Picking (r,c) makes that row and column unavailable.
harvest_grid = [
    [3, 7, 2],
    [4, 5, 6],
    [8, 1, 9]
]

def get_harvest_moves(state):
    # State: (available_cells_tuple, score_A, score_B)
    avail, sa, sb = state
    moves = []
    for (r, c) in avail:
        points = harvest_grid[r][c]
        # New available cells: exclude row r and col c
        new_avail = tuple((nr, nc) for (nr, nc) in avail if nr != r and nc != c)
        # We'll calculate the score later in alpha_beta by looking at the state
        # To fit your function, the next state must include updated scores
        # We track turn using depth in the function
        moves.append(((r, c), (new_avail, points)))
    return moves

# We need a slight wrapper to handle the score accumulation in your signature
def harvest_moves_wrapper(state):
    avail, sa, sb, turn_max = state
    results = []
    for (r, c) in avail:
        pts = harvest_grid[r][c]
        new_avail = tuple((nr, nc) for (nr, nc) in avail if nr != r and nc != c)
        if turn_max:
            results.append(((r,c), (new_avail, sa + pts, sb, False)))
        else:
            results.append(((r,c), (new_avail, sa, sb + pts, True)))
    return results

def harvest_evaluate(state):
    avail, sa, sb, turn = state
    return sa - sb

def harvest_terminal(state):
    avail, sa, sb, turn = state
    return len(avail) == 0

# Initial state: All 9 cells available, scores 0, Farmer A (True) starts
initial_avail = tuple((r, c) for r in range(3) for c in range(3))
initial_state = (initial_avail, 0, 0, True)

val, best_move = alpha_beta(initial_state, 9, -100, 100, True, 
                            harvest_moves_wrapper, harvest_evaluate, harvest_terminal)

print(f"Best harvest move for Farmer A: Cell {best_move}")
print(f"Predicted score difference (A - B): {val}")

### 4. CSP - Backtracking (Map Coloring / Sudoku / Scheduling)

In [ ]:
def csp_solver(assignment, variables, domains, constraints_fn):
    if len(assignment) == len(variables): return assignment
    var = [v for v in variables if v not in assignment][0]
    for val in domains[var]:
        if constraints_fn(var, val, assignment):
            assignment[var] = val
            res = csp_solver(assignment, variables, domains, constraints_fn)
            if res: return res
            del assignment[var]
    return None

# --- SCENARIO EXAMPLE: University Timetable Scheduler ---
courses = ['AI', 'DB']; profs = ['P1', 'P2']
doms = {c: profs for c in courses}
def timetable_constraints(var, val, assign):
    for other_var, other_val in assign.items():
        if val == other_val: return False # Professors can't teach 2 courses at once
    return True

print(f"Schedule: {csp_solver({}, courses, doms, timetable_constraints)}")

## SECTION 3: LOGIC & EXPERT SYSTEMS (Labs 11-12)
### 5. Truth Table Generator (Propositional Logic)

In [ ]:
def truth_table(expression_fn, symbols):
    print(" | ".join(symbols) + " | Result")
    for vals in product([True, False], repeat=len(symbols)):
        args = dict(zip(symbols, vals))
        res = expression_fn(**args)
        print(" | ".join(str(v) for v in vals) + f" | {res}")

# --- SCENARIO EXAMPLE: (P AND Q) OR (NOT R) ---
def my_logic(P, Q, R): return (P and Q) or (not R)
truth_table(my_logic, ['P', 'Q', 'R'])

### 6. Resolution-Based Inference (Medical Expert System)
Simplified expert system logic for Lab 12.

In [ ]:
def expert_system_inference(facts, rules):
    """rules: list of (condition_list, result)"""
    inferred = set(facts)
    changed = True
    while changed:
        changed = False
        for condition, result in rules:
            if all(c in inferred for c in condition) and result not in inferred:
                inferred.add(result); changed = True
    return inferred 

# --- SCENARIO EXAMPLE: Medical Diagnosis ---
kb_rules = [
    (['Fever', 'Cough'], 'COVID'),
    (['Fever'], 'Flu'),
    (['Headache', 'Nausea'], 'Migraine')
]
patient_symptoms = ['Fever', 'Cough']
print(f"Inferred Diagnoses: {expert_system_inference(patient_symptoms, kb_rules)}")

## SECTION 4: MACHINE LEARNING (Labs 13-15)
### 7. Clustering (K-Means / K-Medoids)

In [ ]:
def k_means(data, k, iters=50):
    centroids = data[np.random.choice(data.shape[0], k, replace=False)]
    for _ in range(iters):
        clusters = np.argmin(np.linalg.norm(data[:, None] - centroids, axis=2), axis=1)
        new_centroids = np.array([data[clusters == i].mean(axis=0) for i in range(k)])
        if np.allclose(centroids, new_centroids): break
        centroids = new_centroids
    return clusters, centroids

# --- SCENARIO EXAMPLE: Segmenting Countries by GDP & Literacy ---
### DATASET LOADING TEMPLATE ###
# df = pd.read_csv('countries.csv'); X = df[['GDP', 'Literacy']].values
X_dummy = np.random.rand(20, 2)
clusters, centers = k_means(X_dummy, k=3)
print(f"Cluster assignments for 20 points: {clusters}")

### 8. Neural Rules (Perceptron / Delta Rule / Backprop)

In [ ]:
import numpy as np
class NeuralHelper:
    @staticmethod
    def perceptron(X, y, lr=0.1, epochs=10):
        w = np.zeros(X.shape[1] + 1)
        for _ in range(epochs):
            for xi, target in zip(X, y):
                pred = 1 if (np.dot(xi, w[1:]) + w[0]) >= 0 else 0
                error = target - pred
                w[1:] += lr * error * xi; w[0] += lr * error
        return w

    @staticmethod
    def train_mlp(X, y, h_size=4, lr=0.1, epochs=1000):
        # Simplified MLP for XOR or Iris
        w_h = np.random.rand(X.shape[1], h_size)
        w_o = np.random.rand(h_size, y.shape[1])
        for _ in range(epochs):
            # Forward
            h = 1 / (1 + np.exp(-np.dot(X, w_h)))
            o = 1 / (1 + np.exp(-np.dot(h, w_o)))
            # Backprop
            err_o = (y - o) * (o * (1 - o))
            err_h = err_o.dot(w_o.T) * (h * (1 - h))
            w_o += h.T.dot(err_o) * lr; w_h += X.T.dot(err_h) * lr
        return w_h, w_o
    @staticmethod
    def predict_mlp(X, w_h, w_o):
        """Helper to test the MLP after training"""
        h = 1 / (1 + np.exp(-np.dot(X, w_h)))
        o = 1 / (1 + np.exp(-np.dot(h, w_o)))
        return o

# ==========================================
# HOW TO CALL IN THE EXAM
# ==========================================

# Data for XOR
X_xor = np.array([[0,0], [0,1], [1,0], [1,1]])
y_xor = np.array([[0], [1], [1], [0]])

print("--- EXERCISE 1: PERCEPTRON (For AND Gate) ---")
# Using Perceptron for AND gate (Linearly separable)
y_and = np.array([0, 0, 0, 1])
w_perceptron = NeuralHelper.perceptron(X_xor, y_and)
print(f"Perceptron Weights: {w_perceptron}")
# Testing Perceptron
for x in X_xor:
    res = 1 if (np.dot(x, w_perceptron[1:]) + w_perceptron[0]) >= 0 else 0
    print(f"Input {x} -> Predicted {res}")

print("\n--- EXERCISE 2: MLP (For XOR Gate) ---")
# Training MLP for XOR
wh, wo = NeuralHelper.train_mlp(X_xor, y_xor, h_size=4, lr=0.5, epochs=5000)

# Testing MLP
predictions = NeuralHelper.predict_mlp(X_xor, wh, wo)
print("XOR Predictions (Raw):")
print(predictions)

print("\nXOR Predictions (Rounded):")
for i, x in enumerate(X_xor):
    print(f"Input {x} -> Target {y_xor[i]} -> Predicted {np.round(predictions[i])}")

### 9. Genetic Algorithm (Portfolio Optimization)
Used for Lab 8 Task 2.

In [ ]:
def genetic_opt(pop, fit_fn, mut_fn, cross_fn, gens=50):
    for _ in range(gens):
        pop = sorted(pop, key=fit_fn, reverse=True)
        next_gen = pop[:2] # Elitism
        while len(next_gen) < len(pop):
            p1, p2 = random.sample(pop[:5], 2)
            child = cross_fn(p1, p2)
            if random.random() < 0.2: child = mut_fn(child)
            next_gen.append(child)
        pop = next_gen
    return max(pop, key=fit_fn)

# --- SCENARIO EXAMPLE: Selecting 4 Items (Binary Vector) ---
def fit(ind): return sum(ind) # Goal: max 1s
def cross(p1, p2): return p1[:2] + p2[2:]
def mut(ind): 
    i = random.randint(0, 3); ind[i] = 1 - ind[i]; return ind

start_pop = [[0,0,0,0], [1,1,0,0], [0,0,1,1]]
print(f"Best Chromosome: {genetic_opt(start_pop, fit, mut, cross)}")